Testando se o ambiente está funcionando

In [4]:
print('teste')

teste


Importando bibliotecas para explorar os diretórios do computador

In [5]:
from os import listdir
from os.path import isfile, join

Verificando todos os arquivos obtidos da fonta de dados

In [6]:
path = "E://dump"

dump_files = listdir(path)

In [7]:
import pandas as pd

Tendo certeza que é possível ler os dados do GHtorrent.

In [8]:
#pd.read_csv(join(path, 'project_languages.csv'))

Acredito que será possível partir dos pullrequest para observar quais repositórios possuem contribuições ou não. Além disso, após pesquisar (https://levelup.gitconnected.com/how-to-mine-github-data-in-2022-e9c70b3f61d3) parece que os dados de 2015 possuem uma estrutura diferente. Então irei tentar filtrar eles fora do meu dataset

In [9]:
#df_pullrequests = pd.read_csv(join(path, 'pull_requests.csv'))

In [10]:
#df_pullrequests

Um artigo que mostra a estrutura dos arquivos do GHtorrent https://gousios.org/pub/ghtorrent-dataset-toolsuite.pdf
misturando com esse aqui também https://github.com/ghtorrent/ghtorrent.org/blob/master/files/schema.pdf
Então irei adicionar o nome nas colunas, acabei retirando a coluna pullrequest_id pois ela não fazia sentido no contexto

In [11]:
#df_pullrequests.columns = ["id", "head_repo_id", "base_repo_id", "head_commit_id", "base_commit_id", "intra_branch_id","merged"]

Como os pull requests não possuem data de criação tentarei filtrar através dos commits que eles estão ligados.

In [12]:
# # Função para calcular o número total de chunks
# def get_total_chunks(filename, chunksize):
#     # Contar o número total de linhas no arquivo
#     total_lines = sum(1 for _ in open(filename)) - 1  # Subtrai 1 por causa do cabeçalho
#     # Calcular o número total de chunks
#     total_chunks = (total_lines // chunksize) + (1 if total_lines % chunksize != 0 else 0)
#     return total_chunks

# # Exemplo de uso
# filename = join(path, 'commits.csv')
# chunksize = 5000000
# total_chunks = get_total_chunks(filename, chunksize)
# print(f"Total de chunks: {total_chunks}")

In [13]:
#df_commits_19 = pd.DataFrame({},columns=["id", "sha", "author_id", "commiter_id", "project_id", "created_at"])



In [14]:
#def checkDate(df):
#   return pd.to_datetime(df.iloc[0]['created_at']) > pd.to_datetime('2019-01-01') or pd.to_datetime(df.iloc[-1]['created_at']) > pd.to_datetime('2019-01-01')




In [15]:
# for chunk in df_commits:
#     chunk.map(lambda row: chechRowDate(row))

In [16]:
# i=0

# for chunk in df_commits:
#     if checkDate(chunk):
#         chunk.to_csv(f'E:/dump/commits_greater_than_2k19_{i}')

#     i += 1

Devido a grande quantidade de commits, não foi possível os utilizar como filtro de projetos open-source. Nesse caso, iremos utilizar a relação project_members para encontrar quais projetos são relevantes, ou seja, que possuem contribuintes além do dono.

In [17]:
df_project_members = pd.read_csv(join(path, 'project_members.csv'), names=["repo_id", "user_id", "created_at", "ext_ref_id"])

In [18]:
df_project_members

,repo_id,user_id,created_at,ext_ref_id
0,1,1,2009-12-08 10:17:27,NaN
1,1,2,2009-12-08 10:17:27,NaN
2,1,4,2009-12-08 10:17:27,NaN
3,3,24,2011-02-19 01:58:42,NaN
4,3,5465,2011-02-19 01:58:42,NaN
...,...,...,...,...
15525047,207191305,221488,2021-03-06 22:46:40,NaN
15525048,207191367,54196484,2021-03-06 22:51:54,NaN
15525049,207191455,54204581,2021-03-06 22:50:39,NaN
15525050,207191498,3518100,2021-03-06 22:45:27,NaN


In [19]:
df_count_members = df_project_members.groupby("repo_id", as_index=False)["user_id"].count()

In [18]:
df_count_members

,repo_id,user_id
0,1,3
1,3,19
2,5,1
3,6,6
4,8,16
...,...,...
9549778,207191305,1
9549779,207191367,1
9549780,207191455,1
9549781,207191498,1


In [19]:
df_open_source_members = df_count_members[df_count_members["user_id"]>10]

In [20]:
df_open_source_members

,repo_id,user_id
1,3,19
4,8,16
14,19,14
26,34,38
27,37,42
...,...,...
9544854,206859445,12
9545989,206918002,12
9546379,206955864,17
9547045,206997704,19


Devivo a datação dos dados iremos filtrar somente os membros a partir de 2019

In [24]:
df_project_members = df_project_members.loc[
    (pd.to_datetime(df_project_members['created_at']) > pd.to_datetime("01-01-2019")) & 
    (df_project_members['repo_id'].isin(df_open_source_members['repo_id']))
    ]

In [25]:
df_project_members

,repo_id,user_id,created_at,ext_ref_id
66,19,9245591,2020-06-17 19:28:17,NaN
68,19,33292409,2020-06-05 14:53:56,NaN
1475,346,32145570,2019-01-29 14:55:47,NaN
2199,499,87499,2019-01-10 18:12:44,NaN
2201,499,249763,2019-02-27 17:41:17,NaN
...,...,...,...,...
15524292,207159748,61113108,2021-03-06 17:14:58,NaN
15524293,207159748,61908309,2021-03-06 17:13:48,NaN
15524294,207159748,68016435,2021-03-06 16:52:38,NaN
15524295,207159748,68016479,2021-03-06 16:54:11,NaN


A partir do estudo Recommending GitHub Projects for Developer Onboarding podemos extrair alguns atributos a serem extraídos do dataset para que seja estudado o quão receptivo um repositório é, são eles:

- Crescimento do Projeto: Quantidade de commits do projeto no momento de iniciação
- Data de criação do projeto: Projetos mais novos tendem a atrair mais desenvolvedores
- Momento de primeira contribuição: quando o projeto começa a receber contribuidores
- Momento da última contribuição: se a equipe do projeto se manter estável significa que existe menos oportunidades de onboarding
- Momento do primeiro commit: primeiro commit do projeto, a partir de quando passa a aceitar contribuições
- Momento do último commit: se o projeto receber muitos commits, mas nao tiver tanto onboarding significa q esse projeto necessita de mais contribuintes

In [21]:
#objetivo final

df_onboardings = pd.DataFrame({}, columns=["repo_id", "created_at", "total_commits","first_onboarding", "last_onboarding", "first_commit", "last_commit", "user_id", "onboarding_date"])

df_onboardings

,repo_id,created_at,total_commits,first_onboarding,last_onboarding,first_commit,last_commit,user_id,onboarding_date


In [30]:
chunks_commits = pd.read_csv(join(path, 'commits.csv'), names=["id", "sha", "author_id", "commiter_id", "project_id", "created_at"], chunksize=5000000)

Agora irei filtrar somente os commits dos projetos selecionados como open source após o ano de 2019

In [31]:
#all_open_source_commits = []

i=0
for chunk in chunks_commits:
        df_commit = chunk[chunk['project_id'].isin(df_project_members['repo_id'])]
        df_commit.to_csv(f'E:/dump/open_source/open_source_commits_{i}.csv')
        i += 1

        #all_open_source_commits.append(df_commit)

#df_commits_open_source = pd.concat(all_open_source_commits)

In [1]:
df_commits_open_source

NameError: name 'df_commits_open_source' is not defined

In [ ]:
df_commits_open_source = df_commits_open_source.drop('Unnamed: 0', axis=1)

df_commits_open_source.to_csv(join(path,"df_commits_open_source.csv"))

In [ ]:
#transforma o campo de data em data

df_commits_open_source['created_at'] = pd.to_datetime(df_commits_open_source['created_at'])

In [ ]:
df_commits_open_source

Agora iremos obter o resultado o primeiro commit de cada usuário

In [ ]:
def get_first_user_commit(row):
    df_commits_open_source[df_commits_open_source['author_id'] == row['user_id']]

In [ ]:
unique_users = df_commits_open_source['author_id'].unique()

first_commit_user = []
last_commit_user = []


for user in unique_users:
    df_user_commit = df_commits_open_source[df_commits_open_source['author_id'] == user]
    first_commit_user.append(df_user_commit.iloc[df_user_commit['created_at'].argmin()])
    last_commit_user.append(df_user_commit.iloc[df_user_commit['created_at'].argmax()])

Agora que temos os projetos que possuem contribuidores podemos utilizar para filtrar a relação projetos

In [ ]:
df_first_commit_user = pd.DataFrame(first_commit_user)

df_first_commit_user

Obtemos 26183 primeiras contribuições desde 2019. A partir disso vamos obter os demais dados importantes para contribuições

In [ ]:
repo_done = []

from IPython.display import clear_output

In [ ]:
def get_total_commits(row):
    repo_id = row['project_id']
    date = row['created_at']
    total_commits = 0
    clear_output(wait=True)
    repo_done.append(repo_id)
    print(repo_done)
    for chunk in df_commits:
        total_commits += chunk.loc[(chunk['project_id'] == repo_id) & (pd.to_datetime(chunk['created_at']) <= pd.to_datetime(date))].shape[0]

    return total_commits

In [ ]:

df_first_commit_user['total_commits'] = df_first_commit_user.apply(get_total_commits, axis=1)   

Pegar os detalhes do projeto

In [ ]:
df_projetos = pd.read_csv(join(path, "projects.csv"), names=["id", "url", "owner_id", "name", "descriptor", "language", "created_at", "forked_from", "deleted", "updated_at"], na_values="not available", on_bad_lines='warn', chunksize=5000000)